In [1]:
import os
import pandas as pd
import sys
import regex as re
import numpy as np
import datetime
import time
import snowflake.connector as snow
from ffiec_data_connect import methods, credentials, ffiec_connection
creds = credentials.WebserviceCredentials(username="your-username", password="your-password")
conn = ffiec_connection.FFIECConnection()

In [5]:
def extract_taxonomy_data(df, form_id):
    data_list = []
    for i in range(len(df)):
        mdrm_code = re.split('_',df.index[i])[1]
        keys_list = list(df["data"][i].keys())
        schedule_type = keys_list[0]
        subset_of_data = df["data"][i][keys_list[0]]
        schedule_label = subset_of_data['line_ids']['schedule']['label']
        mdrm_label = subset_of_data['line_ids']['extra_col_0']['label']
        if len(subset_of_data['line_ids']) > 2: 
            j = 1 
            while True:
                try:
                    mdrm_label += subset_of_data['line_ids'][f'extra_col_{j}']['label']
                    j += 1
                except TypeError:
                    j+=1
                    continue
                except KeyError:
                    break
        data_list.append({
            'mdrm_code': mdrm_code,
            'schedule_type': schedule_type,
            'schedule_label': schedule_label,
            'mdrm_label': mdrm_label,
            'form_type': form_id
        })
    final_df = pd.DataFrame(data_list)
    return final_df

In [ ]:
def return_id_rssd():
    return id_rssd_set

In [6]:
def match_data_type(id_rssd_set, mdrm_set):
    master_data_type_list = []
    trials = 0
    print("The time of code execution begin is : ", time.ctime())
    while len(mdrm_set) != 0 and trials < len(id_rssd_set):
        for id_rssd in id_rssd_set:
                time_series = methods.collect_data(
                session=conn,
                creds=creds,
                rssd_id=id_rssd,
                reporting_period="9/30/2023",
                series="call",
                output_type='pandas'
                )
                for mdrm in mdrm_set.copy():
                    if mdrm in time_series['mdrm'].values:
                        data_type = time_series['data_type'][time_series['mdrm']==mdrm].values[0]
                        pair_mdrm_type = (mdrm,data_type)
                        master_data_type_list.append(pair_mdrm_type)
                        mdrm_set.remove(mdrm)
                trials +=1
                if not mdrm_set:
                    break
                time.sleep(1.5) #so we don't run over limit
    print("The time of code execution end was : ", time.ctime())
    return master_data_type_list

In [1]:
def filter_mdrm(path):
    not_mnemonic= ['TEXT', 'TE01', 'TE02', 'TE03','TE04','TE05','TE06','TE07','TE08','TE09','TE10','TE11','TE12']
    in_report = ['FFIEC 031', 'FFIEC041','FFIEC 051']
    mdrm_csv = pd.read_csv(path, skiprows=1)
    mdrm_csv.insert(2, 'mdrm_code', mdrm_csv['Mnemonic'] + mdrm_csv['Item Code'])
    mdrm_filtered = mdrm_csv[(mdrm_csv['Reporting Form'].isin(in_report)) & (~mdrm_csv['Mnemonic'].isin(not_mnemonic))]
    mdrm_filtered.rename(columns={"Reporting Form": "form_type"},inplace=True)
    return mdrm_filtered

In [5]:
def create_table_query(table_name, columns_dict):
    columns = list(columns_dict.keys())
    data_types = list(columns_dict.values())

    if len(columns) != len(data_types):
        raise ValueError("Number of columns and data types must be the same.")

    columns_str = ', '.join([f'{col} {data_type}' for col, data_type in zip(columns, data_types)])
    query = f'CREATE TABLE IF NOT EXISTS {table_name} ({columns_str}, primary key (IDRSSD, RCON9999));'

    return query

In [6]:
def gather_all_queries(final_df):
    create_queries_list= []
    schedules = set(final_df['schedule_type'])
    for s in schedules:
        schedule_dict_list = {sd['mdrm_code']: sd['data_type'] for sd in final_df.loc[final_df['schedule_type'].isin([s]), 
                              ['mdrm_code', 'data_type']].to_dict(orient='records')}
        bank_id = 'IDRSSD'
        bank_id_data_type = 'VARCHAR'
        report_date = 'RCON9999'
        report_date_data_type= 'DATE'
        for key,item in schedule_dict_list.items():
            if item == 'str':
                schedule_dict_list[key] = 'VARCHAR'
            elif item in ['float','int']:
                schedule_dict_list[key] = 'FLOAT'
            elif item == 'bool':
                schedule_dict_list[key] = 'BOOLEAN'   
        if bank_id in schedule_dict_list:
            my_dict[bank_id] = bank_id_data_type
        else:
            schedule_dict_list[bank_id] = bank_id_data_type
        if report_date in schedule_dict_list:
            schedule_dict_list[report_date] = report_date_data_type
        else:
            schedule_dict_list[report_date] = report_date_data_type
        create_queries_list.append(create_table_query(f'Schedule_{s}', schedule_dict_list))
    return create_queries_list

In [11]:
conn= snow.connect(user="your-username", password="your-password", account="your-account")

In [9]:
def main(conn, full_query_list):
    mdrm_filtered = filter_mdrm(path="C:\\Users\\whill\\call-report-database\\MDRM_CSV.csv")
    cur = conn.cursor()
    sql = "USE ROLE ACCOUNTADMIN"
    cur.execute(sql)
    sql = "USE WAREHOUSE COMPUTE_WH"
    cur.execute(sql)
    sql = "ALTER WAREHOUSE COMPUTE_WH RESUME IF SUSPENDED"
    cur.execute(sql)
    sql = "CREATE DATABASE IF NOT EXISTS CALLREPORTDB"
    cur.execute(sql)
    sql = "USE DATABASE CALLREPORTDB"
    cur.execute(sql)
    sql = "CREATE SCHEMA IF NOT EXISTS PUBLIC"
    cur.execute(sql)
    sql = "USE SCHEMA PUBLIC"
    cur.execute(sql)
    for query in full_query_list:
        cur.execute(query)
    sql = "ALTER WAREHOUSE COMPUTE_WH SUSPEND"
    cur.execute(sql)
    cur.close()
    conn.close()

In [12]:
if __name__ == "__main__":
    main(conn,full_query_list)

In [ ]:
mdrm_filtered = filter_mdrm(path="C:\\Users\\whill\\call-report-database\\MDRM_CSV.csv")

In [5]:
filers = methods.collect_filers_on_reporting_period(
    session=conn,
    creds=creds,
    reporting_period="9/30/2023",
    output_type='pandas'
)
filers

,id_rssd,fdic_cert_number,occ_chart_number,ots_dock_number,primary_aba_rout_number,name,state,city,address,filing_type,has_filed_for_reporting_period
0,37,10057,None,16553,61107146,BANK OF HANCOCK COUNTY,GA,SPARTA,12855 BROAD STREET,051,True
1,242,3850,None,None,81220537,FIRST COMMUNITY BANK XENIA-FLORA,IL,XENIA,260 FRONT STREET,051,True
2,279,28868,None,2523,311972526,"MINEOLA COMMUNITY BANK, SSB",TX,MINEOLA,215 W BROAD,051,True
3,354,14083,None,None,101107475,BISON STATE BANK,KS,BISON,223 MAIN STREET,051,True
4,457,10202,None,None,91208332,LOWRY STATE BANK,MN,LOWRY,400 FLORENCE AVE.,051,True
...,...,...,...,...,...,...,...,...,...,...,...
4664,5805479,59346,25287,None,122402463,"BESSEMER TRUST COMPANY OF NEVADA, NATIONAL ASS...",NV,LAS VEGAS,1700 SOUTH PAVILION CENTER DRIVE SUITE...,051,True
4665,5805488,59349,25275,None,121202457,"INSPIRE TRUST COMPANY, NATIONAL ASSOCIATION",NV,RENO,241 RIDGE STREET SUITE 310,041,True
4666,5805817,59337,25237,None,91018470,CERIDIAN NATIONAL TRUST BANK,MN,BLOOMINGTON,3311 EAST OLD SHAKOPEE ROAD,051,True
4667,5859511,59344,25288,None,53012977,"TIAA TRUST, NATIONAL ASSOCIATION",NC,CHARLOTTE,8500 ANDREW CARNEGIE BOULEVARD,051,True


In [6]:
id_rssd_set = set(filers['id_rssd'])

In [ ]:
df1 = extract_taxonomy_data(pd.read_json("C:\\Users\\whill\\call-report-database\\python\\cdr_taxonomy_xbrl_to_json\\example\\031_2023-09-30.json")
                            ,'FFIEC 031')
df2 = extract_taxonomy_data(pd.read_json("C:\\Users\\whill\\call-report-database\\python\\cdr_taxonomy_xbrl_to_json\\example\\041_2023-09-30.json")
                            ,'FFIEC 041')
df3 = extract_taxonomy_data(pd.read_json("C:\\Users\\whill\\call-report-database\\python\\cdr_taxonomy_xbrl_to_json\\example\\051_2023-09-30.json")
                            ,'FFIEC 051')
merged_df = pd.concat([df1,df2,df3]).reset_index(drop=True)
#merged_df.to_csv("C:\\Users\\whill\\call-report-database\\python\\cdr_taxonomy_xbrl_to_json\\example\\Taxonomies_example.csv")
df1 = extract_taxonomy_data(pd.read_json("C:\\Users\\whill\\call-report-database\\python\\cdr_taxonomy_xbrl_to_json\\example\\031_2001-03-31.json")
                            ,'FFIEC 031')
df2 = extract_taxonomy_data(pd.read_json("C:\\Users\\whill\\call-report-database\\python\\cdr_taxonomy_xbrl_to_json\\example\\041_2001-03-31.json")
                            ,'FFIEC 041')
older_df = pd.concat([df1,df2]).reset_index(drop=True)
#All Taxonomy data is merged now match with MDRM file so we can have the unique identifier and which schedule 
df_2000_2023 = pd.concat([merged_df,older_df]).reset_index(drop=True)
df_2000_2023.drop_duplicates(subset=['mdrm_code', 'form_type'],keep='first',inplace=True,ignore_index=True)
rows_to_drop = np.where(df_2000_2023['mdrm_code'].str.contains('TE', regex=True))# TE/TEXT reporting items are of limited value and sparse 
df_2000_2023.drop(index=rows_to_drop[0],inplace =True)
result_df = mdrm_filtered.join(df_2000_2023.set_index(['mdrm_code', 'form_type']), on=['mdrm_code','form_type'],how='left')
#result_df[~result_df['schedule_type'].isnull()] result_df.to_csv("C:\\Users\\whill\\call-report-database\\MDRM_CSV_Filtered_Schedules.csv")
master_data_type_list = match_data_type(id_rssd_set,set(result_df['mdrm_code']))
datatype_df = pd.DataFrame(master_data_type_list,columns=['mdrm_code', 'data_type'])
final_df = result_df.join(datatype_df.set_index(['mdrm_code']), on=['mdrm_code'],how='left')

#Set undiscovered Scheduled to Undefined and define remaining data types based on item type descriptions
final_df.loc[(final_df['Data_Type'].isnull()) & ((final_df['Item_Type']=='D') | (final_df['Item_Type']=='R')), 'Data_Type']='float'
final_df.loc[(final_df['Data_Type'].isnull()) & ((final_df['Item_Type']=='F') | (final_df['Item_Type']=='S')), 'Data_Type']='str'
final_df.loc[final_df['Schedule_Type'].isnull(),'Schedule_Type']='Undefined'
final_df.to_csv("C:\\Users\\whill\\OneDrive\\Desktop\\Fall2022\\ISYE6740\\call-report-database\\MDRM_CSV_Filtered_Schedules_Data_Types.csv", index=False)

In [7]:
full_query_list = gather_all_queries(final_df)

In [4]:
final_df = pd.read_csv("C:\\Users\\whill\\OneDrive\\Desktop\\Fall2022\\ISYE6740\\call-report-database\\MDRM_CSV_Filtered_Schedules_Data_Types.csv")
final_df

,Mnemonic,Item_Code,MDRM_Code,Start_Date,End_Date,Item_Name,Confidentiality,Item_Type,Form_Type,Description,Series_Glossary,Schedule_Type,Schedule_Label,MDRM_Label,Data_Type
0,CALL,8002,CALL8002,1984-03-31,2006-03-31,REPORTING STATUS (W-CODE),N,F,FFIEC 031,This item is an asset threshold used to determ...,\r\nThe following items may be accessed by the...,NaN,NaN,NaN,NaN
1,CALL,8786,CALL8786,1984-03-31,9999-12-31,REPORTING LEVEL CODE,N,S,FFIEC 031,Effective 10/24/2011; the FR 2572 information ...,\r\nThe following items may be accessed by the...,NaN,NaN,NaN,NaN
2,CALL,8787,CALL8787,1959-12-31,9999-12-31,REPORTING EXCEPTION CODE,N,F,FFIEC 031,Effective 10/24/2011; the FR 2572 information ...,\r\nThe following items may be accessed by the...,NaN,NaN,NaN,NaN
3,CALL,8798,CALL8798,1959-12-31,9999-12-31,FDR TIME STAMP,Y,F,FFIEC 031,Effective 10/24/2011; the FR 2572 information ...,\r\nThe following items may be accessed by the...,NaN,NaN,NaN,NaN
4,CALL,8799,CALL8799,1959-12-31,2000-12-31,ARCHIVAL DATE STAMP,Y,F,FFIEC 031,The date (ccyymmdd) the data were last updated...,\r\nThe following items may be accessed by the...,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6057,RSSD,9130,RSSD9130,2017-03-31,9999-12-31,CITY/TOWN TEXT NAME,N,S,FFIEC 051,The name of the city or town in which the enti...,,ENT,Bank Demographic Information(Form Type - 051),City,str
6058,RSSD,9200,RSSD9200,2005-09-30,9999-12-31,ABBREVIATED STATE NAME,N,S,FFIEC 031,Two character state abbreviation as assigned t...,,ENT,Bank Demographic Information(Form Type - 031),State abbreviation,str
6059,RSSD,9200,RSSD9200,2017-03-31,9999-12-31,ABBREVIATED STATE NAME,N,S,FFIEC 051,Two character state abbreviation as assigned t...,,ENT,Bank Demographic Information(Form Type - 051),State abbreviation,str
6060,RSSD,9220,RSSD9220,2005-09-30,9999-12-31,PHYSICAL ZIP/FOREIGN MAILING CODE,N,S,FFIEC 031,The zip code or foreign mailing code of the ph...,,ENT,Bank Demographic Information(Form Type - 031),Zip code,str


## lets define the last of our data types based on the item types provided for us R- rate stored as a decimal so obviously we need float, for easy we are going to define D and R as float, I went and checked the 443 remaining blank for D and checked that they would be mostly numerical. Then we will set everything to str for remain F and S codes, finally we will set the last schedule as undefined!

In [7]:
final_df.loc[(final_df['Data_Type'].isnull()) & ((final_df['Item_Type']=='D') | (final_df['Item_Type']=='R')), 'Data_Type']='float'
final_df.loc[(final_df['Data_Type'].isnull()) & ((final_df['Item_Type']=='F') | (final_df['Item_Type']=='S')), 'Data_Type']='str'
final_df.loc[final_df['Schedule_Type'].isnull(),'Schedule_Type']='Undefined'

In [14]:
start_list = [('IDRSSD','VARCHAR'), ('RCON9999','DATE')]
start_list.extend(final_df['mdrm_code'][final_df['schedule_type'].isin(['ENT'])].to_list())
columns_set = set(start_list)
columns_set

,Mnemonic,Item Code,mdrm_code,Start Date,End Date,Item Name,Confidentiality,ItemType,form_type,Description,SeriesGlossary,schedule_type,schedule_label,mdrm_label,data_type
0,CALL,8002,CALL8002,3/31/1984 0:00,3/31/2006 0:00,REPORTING STATUS (W-CODE),N,F,FFIEC 031,This item is an asset threshold used to determ...,\r\nThe following items may be accessed by the...,Undefined,NaN,NaN,str
1,CALL,8786,CALL8786,3/31/1984 0:00,12/31/9999 0:00,REPORTING LEVEL CODE,N,S,FFIEC 031,Effective 10/24/2011; the FR 2572 information ...,\r\nThe following items may be accessed by the...,Undefined,NaN,NaN,str
2,CALL,8787,CALL8787,12/31/1959 0:00,12/31/9999 0:00,REPORTING EXCEPTION CODE,N,F,FFIEC 031,Effective 10/24/2011; the FR 2572 information ...,\r\nThe following items may be accessed by the...,Undefined,NaN,NaN,str
3,CALL,8798,CALL8798,12/31/1959 0:00,12/31/9999 0:00,FDR TIME STAMP,Y,F,FFIEC 031,Effective 10/24/2011; the FR 2572 information ...,\r\nThe following items may be accessed by the...,Undefined,NaN,NaN,str
4,CALL,8799,CALL8799,12/31/1959 0:00,12/31/2000 0:00,ARCHIVAL DATE STAMP,Y,F,FFIEC 031,The date (ccyymmdd) the data were last updated...,\r\nThe following items may be accessed by the...,Undefined,NaN,NaN,str
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6057,RSSD,9130,RSSD9130,3/31/2017 0:00,12/31/9999 0:00,CITY/TOWN TEXT NAME,N,S,FFIEC 051,The name of the city or town in which the enti...,,ENT,Bank Demographic Information(Form Type - 051),City,str
6058,RSSD,9200,RSSD9200,9/30/2005 0:00,12/31/9999 0:00,ABBREVIATED STATE NAME,N,S,FFIEC 031,Two character state abbreviation as assigned t...,,ENT,Bank Demographic Information(Form Type - 031),State abbreviation,str
6059,RSSD,9200,RSSD9200,3/31/2017 0:00,12/31/9999 0:00,ABBREVIATED STATE NAME,N,S,FFIEC 051,Two character state abbreviation as assigned t...,,ENT,Bank Demographic Information(Form Type - 051),State abbreviation,str
6060,RSSD,9220,RSSD9220,9/30/2005 0:00,12/31/9999 0:00,PHYSICAL ZIP/FOREIGN MAILING CODE,N,S,FFIEC 031,The zip code or foreign mailing code of the ph...,,ENT,Bank Demographic Information(Form Type - 031),Zip code,str


# break

# just set things to 'Missing' Schedule Name, work on developing the code for creating the tables and setting up a new snowflake instance, then work on code for pulling the data merging it together by bank and then looping through the mdrm_code and tying the value to a specific table. 

# we also need to think about the types of the values, see if we can loop through a list of bank's first, find a data type and then create logic to develop the create table queries 

In [14]:
result_df.to_csv("C:\\Users\\whill\\call-report-database\\MDRM_CSV_Filtered_Schedules.csv")

In [8]:
merged_df.to_csv("C:\\Users\\whill\\call-report-database\\python\\cdr_taxonomy_xbrl_to_json\\example\\Taxonomies_example.csv")